In [131]:
import angr
import pyvex
import archinfo
from angrutils import *
import networkx as nx
import claripy
import matplotlib.pyplot as plt

In [132]:
# verified
# removes call and return edges of the cfg
class newCfg:
    def __init__(self,cfg):
        self.g = nx.DiGraph()
        i = 0
        nodeList = list(cfg.graph.nodes)
        for n in nodeList:
            self.g.add_node(n.addr)
            for nei in cfg.graph.adj[n]:
                j = cfg.graph.adj[n][nei]['jumpkind']
                if  j == 'Ijk_Call':
                    pass
                elif j == 'Ijk_Ret':
                    pass
                elif j == 'Ijk_Exit':
                    pass
                elif j == 'Ijk_FakeRet':
                    self.g.add_edge(n.addr,nei.addr)
                elif j == 'Ijk_Boring':
                    self.g.add_edge(n.addr,nei.addr)
                else :
                    raise Exception(j)
                    
    def prnt(self):
        for n in self.g.nodes:
            s = hex(n)
            s+= ': '
            for n1 in self.g.adj[n]:
                s+= hex(n1)
                s+= ' '
            print(s)
    

In [133]:
# verified
# gives the value of claripy bitvector 
def get_val(x):
    size = x.args[1]
    size = 2**(size)
    val = x.args[0]
    if val >= size/2:
        val = val-size
    return val

In [134]:
# partially verified
# given a block constructs a flow graph


class block_state:
    def __init__(self,b):
        self.g = nx.DiGraph()
        self.stateMap = {"esp":(1,claripy.BVV(0x0,64)),"ebp":(2,claripy.BVV(0x0,64)),"addr":None}
        self.tempVals = {}
        self.timeMap = {}
        self.used = []
        self.killed = []
        self.g.add_node("start")
        self.funcall = None
        
        try:
            stmts = b.vex.statements
            if b.vex.jumpkind == 'Ijk_Call':
                self.funcall = list(b.vex.constant_jump_targets)[0]
        except:
            stmts = []
        
        inst_no = 0 
        for stmt in stmts:
            self.update(stmt,inst_no)
            inst_no+=1

        
    def update(self,stmt,inst_no):
        
        self.used = []
        self.killed = []
        
        if stmt.tag == 'Ist_NoOp':
            self.__NoOp(stmt)
        if stmt.tag == 'Ist_IMark':
            self.__IMark(stmt)
        if stmt.tag == 'Ist_AbiHint':
            self.__AbiHint(stmt)
        if stmt.tag == 'Ist_Put':
            self.__Put(stmt)
        if stmt.tag == 'Ist_PutI':
            self.__PutI(stmt)
        if stmt.tag == 'Ist_WrTmp':
            self.__WrTmp(stmt)
        if stmt.tag == 'Ist_Store':
            self.__Store(stmt)
        if stmt.tag == 'Ist_CAS':
            self.__CAS(stmt)
        if stmt.tag == 'Ist_LLSC':
            self.__LLSC(stmt)
        if stmt.tag == 'Ist_MBE':
            self.__MBE(stmt)
        if stmt.tag == 'Ist_Dirty':
            self.__Dirty(stmt)
        if stmt.tag == 'Ist_Exit':
            self.__Exit(stmt)
        if stmt.tag == 'Ist_LoadG':
            self.__LoadG(stmt)
        if stmt.tag == 'Ist_StoreG':
            self.__StoreG(stmt)
            
        self.upateGraph(inst_no)
        
    def upateGraph(self,inst_no): # verified
        
        g = self.g
                
        for (typ,offset) in self.used:
            if typ == 'stack' or typ == 'glb' or typ == 'base':
                offset = get_val(offset)
                
            if typ == 'mem':
                if 'mem' in g.nodes:
                    g.add_edge("mem",inst_no)
                else :
                    g.add_edge("start","mem")
                    g.add_edge("mem",inst_no)
            else:
                if (typ,offset) in self.timeMap:
                    n = (typ,offset,self.timeMap[(typ,offset)])
                    g.add_edge(n,inst_no)
                else:
                    self.timeMap[(typ,offset)] = 0
                    g.add_edge((typ,offset,0),inst_no)
                    g.add_edge("start",(typ,offset,0))
        
        for (typ,offset) in self.killed:
            if typ == 'stack' or typ == 'glb' or typ == 'base':
                offset = get_val(offset)
                
            u = (typ,offset)
            if typ == 'mem':
                g.add_edge(inst_no,"mem")
            else:
                if u in self.timeMap:
                    t = self.timeMap[u]
                    self.timeMap[u] = t+1
                    t = t+1
                    g.add_edge(inst_no,(typ,offset,t))
                else:
                    self.timeMap[u] = 1
                    g.add_edge(inst_no,(typ,offset,1))
        
        
    def __NoOp(self,stmt):
        pass

    def __IMark(self,stmt):
        self.stateMap['addr'] = stmt.addr

    def __AbiHint(self,stmt):
        pass

    def __Put(self,stmt):
        v = self.evalexpr(stmt.data)
        if stmt.offset == 56:
            self.stateMap['esp'] = v
            
        if stmt.offset == 48:
            self.stateMap['ebp'] = v
        
        if stmt.offset != 184:
            self.killed.append(("guest",stmt.offset))
        

    def __PutI(self,stmt):
        raise Exception('Not Implimented PutI')

    def __WrTmp(self,stmt):
        t = stmt.tmp
        v = self.evalexpr(stmt.data)
        self.tempVals[t] = v
        self.killed.append(("temp",t))

    def __Store(self,stmt):
        ind = self.evalexpr(stmt.addr)
        val = self.evalexpr(stmt.data)
#         print(ind)
        if ind == None:
            self.killed.append(("mem",None))
        
        elif ind[0] == 0:
            self.killed.append(("glb",ind[1]))
            
        elif ind[0] == 1:
            self.killed.append(("stack",ind[1]))
            
        elif ind[0] == 2:
            self.killed.append(("base",ind[1]))

    def __CAS(self,stmt):  # compare and swap
        raise Exception('Not Implimented CAS')

    def __LLSC(self,stmt): # Either Load-Linked or Store-Conditional
        raise Exception('Not Implimented LLSC')

    def __MBE(self,stmt):  # memory bus event
        raise Exception('Not Implimented MBE')

    def __Dirty(self,stmt):
        raise Exception('Not Implimented Dirty')

    def __Exit(self,stmt):
        pass
#         raise Exception('Not Implimented Exit')

    def __LoadG(self,stmt):
        raise Exception('Not Implimented LoadG')

    def __StoreG(self,stmt):
        raise Exception('Not Implimented StoreG')
        
    def evalexpr(self,expr):
        if expr.tag == 'Iex_Binder':
            return self.__Binder(expr)
        if expr.tag == 'Iex_VECRET':
            return self.__VECRET(expr)
        if expr.tag == 'Iex_GSPTR':
            return self.__GSPTR(expr)
        if expr.tag == 'Iex_GetI':
            return self.__GetI(expr)
        if expr.tag == 'Iex_RdTmp':
            return self.__RdTmp(expr)
        if expr.tag == 'Iex_Get':
            return self.__Get(expr)
        if expr.tag == 'Iex_Qop':
            return self.__Qop(expr)
        if expr.tag == 'Iex_Triop':
            return self.__Triop(expr)
        if expr.tag == 'Iex_Binop':
            return self.__Binop(expr)
        if expr.tag == 'Iex_Unop':
            return self.__Unop(expr)
        if expr.tag == 'Iex_Load':
            return self.__Load(expr)
        if expr.tag == 'Iex_Const':
            return self. __Const(expr)
        if expr.tag == 'Iex_ITE':
            return self.__ITE(expr)
        if expr.tag == 'Iex_CCall':
            return self.__CCall(expr)
    
    def __Binder(self,expr):
        raise Exception('Not Implimented Binder')

    def __VECRET(self,expr):
        raise Exception('Not Implimented VECRET')

    def __GSPTR(self,expr):
        raise Exception('Not Implimented GSPTR')

    def __GetI(self,expr):
        raise Exception('Not Implimented GetI')

    def __RdTmp(self,expr):
        self.used.append(("temp",expr.tmp))
        if expr.tmp in self.tempVals:
            return self.tempVals[expr.tmp]
        else:
            raise Exception("unassigned Temp")

    def __Get(self,expr):
        if expr.offset != 184:
            self.used.append(("guest",expr.offset))
        
        if expr.offset == 184:
            return self.stateMap['addr']
        if expr.offset == 56:
            return self.stateMap['ebp']
        if expr.offset == 48:
            return self.stateMap['esp']
        
        return None

    def __Qop(self,expr):
        l = []
        for arg in expr.args:
            l.append(self.evalexpr(arg))
        raise Exception('Not Implimented Qop')

    def __Triop(self,expr):
        l = []
        for arg in expr.args:
            l.append(self.evalexpr(arg))
        raise Exception('Not Implimented Triop')

    def __Binop(self,expr):
        l = []
        for arg in expr.args:
            l.append(self.evalexpr(arg))
        if l[0] == None or l[1] == None:
            return None
        
        if expr.op == 'Iop_Sub64':
            if (l[0][0]==1 and l[1][0] == 1):
                return (0,l[0][1]-l[1][1])
            elif (l[0][0]==2 and l[1][0] == 2):
                return (0,l[0][1]-l[1][1])
            elif(l[0][0]==0 and l[1][0] == 0):
                return (0,l[0][1]-l[1][1])
            elif(l[0][0]==1 and l[1][0] == 0):
                return (1,l[0][1]-l[1][1])
            elif(l[0][0]==2 and l[1][0] == 0):
                return (2,l[0][1]-l[1][1])
            else:
                return None
            
        if expr.op == 'Iop_Add64':
            if (l[0][0]==1 and l[1][0] == 1):
                return None
            elif (l[0][0]==2 and l[1][0] == 2):
                return None
            elif(l[0][0]==0 and l[1][0] == 0):
                return (0,l[0][1]+l[1][1])
            elif(l[0][0]==1 and l[1][0] == 0):
                return (1,l[0][1]+l[1][1])
            elif(l[0][0]==0 and l[1][0] == 1):
                return (1,l[0][1]+l[1][1])
            elif(l[0][0]==2 and l[1][0] == 0):
                return (2,l[0][1]+l[1][1])
            elif(l[0][0]==0 and l[1][0] == 2):
                return (2,l[0][1]+l[1][1])
        
        raise Exception(expr.op)

    def __Unop(self,expr):
        v = self.evalexpr(expr.args[0])
        if v == None:
            return None
        
        if expr.op == "Iop_64to32":
            if v[0] != 0:
                return None
            else :
                return (0,v[1].chop(32)[1])
            
        if expr.op == "Iop_32to64":
            return (v[0],v[1].sign_extend(32))
            
        raise Exception(expr.op)

    def __Load(self,expr):
        v = self.evalexpr(expr.addr)
        if v == None:
            self.used.append(("mem",None))
        
        elif v[0] == 1:
            self.used.append(("stack",v[1]))
            
        elif v[0] == 2:
            self.used.append(("base",v[1]))
        
        elif v[0] == 0:
            self.used.append(("glb",v[1]))
        
        return None

    def __Const(self,expr):
#         print("yes)
        if expr.con.type == 'Ity_I64':
            return (0,claripy.BVV(expr.con.value,64))
        
        if expr.con.type == 'Ity_I32':
            return (0,claripy.BVV(expr.con.value,32))
        
        if expr.con.type == 'Ity_I8':
            return (0,claripy.BVV(expr.con.value,8))
        
        raise Exception(expr.con.type)
        
    def __ITE(self,expr):
        raise Exception('Not Implimented ITE')

    def __CCall(self,expr):
        l = []
        for arg in expr.args:
            l.append(self.evalexpr(arg))
        raise Exception('Not Implimented CCall')
    
    def print_args(self):
        for s in self.g.successors('start'):
            print(s)

In [135]:
# verified
# finds graphs for eaxh blocks and stores it in a map

def find_block_state(cfg):
    m = {}
    for n in cfg.graph.nodes:
        if n.addr in m:
            pass
        else:
            m[n.addr] = block_state(n.block)
    return m

In [136]:
# verified
# processes graphs of each block and stores in map form

class compress_state:
    def __init__(self):
        self.m = {}                       # map bw variables and set of variables it can reach at end of function
        self.chng = set()                 # Killed vareables in block
        self.esp = (1,claripy.BVV(0x0,64))# esp value after exicuting block
        self.ebp = (2,claripy.BVV(0x0,64))# ebp value after exicuting block
        self.funcall = None               # addr of function call if present
        
    def find_reachable(self,n,s,vis,ends):
        vis.add(n)
        for v in s.g.successors(n):
            if v not in vis:
                if v in self.chng:
                    ends.add((v[0],v[1]))
                self.find_reachable(v,s,vis,ends)
                
        
    def construct(self,s):
        for p in s.timeMap:
            if p[0] != 'temp' and s.timeMap[p]!=0:
                self.chng.add((p[0],p[1],s.timeMap[p]))
        
        self.esp = s.stateMap['esp']
        if self.esp != None:
            if self.esp[0] == 1:
                self.esp = ('stack',self.esp[1])
            elif self.esp[0] == 2:
                self.esp = ('base',self.esp[1])
            else:
                raise Exception("unexpected value in esp")
                    
        self.ebp = s.stateMap['ebp']
        if self.ebp != None:
            if self.ebp[0] == 1:
                self.ebp = ('stack',self.ebp[1])
            elif self.ebp[0] == 2:
                self.ebp = ('base',self.ebp[1])
            else:
                raise Exception("unexpected value in ebp")
                    
        self.funcall = s.funcall
        
        for n in s.g.successors('start'):
            vis = set()
            ends = set()
            self.find_reachable(n,s,vis,ends)
            self.m[(n[0],n[1])] = ends
        
        s = set()
        for (a,b,c) in self.chng:
            s.add((a,b))
        self.chng = s

In [137]:
# verified

def find_compress_block_state(m):
    cm = {}
    for addr in m:
        c = compress_state()
        c.construct(m[addr])
        cm[addr] = c
    return cm

In [138]:
# verified

class myBlock:
    def __init__(self):
        self.changed = False
        
        self.parMap = {}
        self.blockMap = {}
        self.updatedBlockMap = {}
        self.outMap = {}
        
        self.parShouldKill = None
        self.parMayKill = set()
        self.outMayKill = set()
        self.outShouldKill = None
        self.shouldKill = set()
        self.updatedshouldKill = set()
        
        self.parEspOff = set()
        self.parEbpOff = set()
        self.outEspOff = set()
        self.outEbpOff = set()
        self.espOff = set()
        self.ebpOff = set()
        
        self.funcall = None
    
    def cons(self,c):
        self.blockMap = c.m
        self.shouldKill = c.chng
        if c.esp != None:
            self.espOff.add(c.esp)
        if c.ebp != None:
            self.ebpOff.add(c.ebp)
        self.funcall = c.funcall

In [139]:
# verified

def generate_myBlock(cm):
    out = {}
    for addr in cm:
        mb = myBlock()
        mb.cons(cm[addr])
        out[addr] = mb
    return out

In [140]:
class funDet:
    def __init__(self):
        self.m = {}
        self.shouldKill = None

In [141]:
# verified

def op1(par,chi):
    
    if par == None:
        return (chi,False)
    
    flag = False
    if chi:
        chi = chi.copy()
        for k in par:
            if k in chi:
                s = chi[k] | par[k]
                if not(s == chi[k]):
                    flag = True
                    chi[k] = s
            else:
                chi[k] = par[k].copy()
                flag = True
    else:
        if par:
            chi = par.copy()
            flag = True
        
    return (chi,flag)

In [142]:
# verified

def op2(par,chi,parkilled,chikilled):
    if par == None:
        return chi
    
    if chi == None:
        return par
    
    m = par.copy()
    for k1 in par:
        s1 = set()
        s2 = set()
        for k2 in chi:
            if k2 in par[k1]:
                if (chikilled!=None ) and (k2 in chikilled):
                    s1.add(k2)
                s2 |= chi[k2]
                
        m[k1] = m[k1] - s1
        m[k1] |= s2
    
    for k2 in chi:
        if (parkilled) == None or (k2 not in parkilled):
            if k2 in m:
                m[k2] = m[k2] | chi[k2]
            else:
                m[k2] = chi[k2]
        
    return m

In [143]:
# verified 
def relax(addr,naddr,mbm):
    
    p = mbm[addr]
    c = mbm[naddr]
    
    (c.parMap,c.changed) = op1(p.outMap,c.parMap)
        
    if p.outShouldKill:
        if c.parShouldKill:
            s = c.parShouldKill & p.outShouldKill
            if not(c.parShouldKill == s):
                c.parShouldKill = s
                c.changed = True
        else:
                c.parShouldKill = p.outShouldKill.copy()
                c.changed = True
        
    s = c.parEspOff | p.outEspOff
    if not(s == c.parEspOff):
        c.parEspOff =  s
        c.changed = True
        
    s = c.parEbpOff | p.outEbpOff
    if not(s == c.parEbpOff):
        c.parEbpOff =  s
        c.changed = True
                    

In [144]:
# verified
def updateSet(s,espoff,ebpoff,bit_rep = 0):
    ans = set()
    if s == None:
        return None
    for k in s:
        if k[0] == 'stack':
            s1 = updatesingle(k,espoff,bit = bit_rep)
            ans = ans | s1
        elif k[0] == 'base':
            s1 = updatesingle(k,ebpoff,bit = bit_rep)
            ans = ans | s1
        else:
            ans.add(k)
    
    return ans
        

In [145]:
# verified
def updatesingle(k,off,bit = 0):
    
    if off == None:
        return set()
    
    s = set()
    for x in off:
        try:
            su = x[1]+k[1]
            num = 2**(su.args[1])
            b = su.args[1]
            su = su.args[0]
            
            if su > num/2:
                su = su - num
            if bit == 0: 
                s.add((x[0],su))
            else:
                s.add((x[0],claripy.BVV(su,b)))
        except:
            raise Exception(k,off)
            
    return s

In [146]:
# partially verified looks fine

def updateBlockMap(mbm,addr):
    
    mbm[addr].updatedBlockMap = {}
    if mbm[addr].shouldKill == None:
        mbm[addr].shouldKill = None
    else:
        mbm[addr].updatedshouldKill = updateSet(mbm[addr].shouldKill,mbm[addr].parEspOff,mbm[addr].parEbpOff)
    
    for k in mbm[addr].blockMap:
        if k[0] == 'stack':
            s1 = updatesingle(k,mbm[addr].parEspOff)
        elif k[0] == 'base':
            s1 = updatesingle(k,mbm[addr].parEbpOff)
        else:
            s1 = set()
            s1.add(k)
            
        s2 = updateSet(mbm[addr].blockMap[k],mbm[addr].parEspOff,mbm[addr].parEbpOff)
        
        for k2 in s1:
            if k2 in mbm[addr].updatedBlockMap:
                mbm[addr].updatedBlockMap[k2] = mbm[addr].updatedBlockMap[k2] | s2
            else :
                mbm[addr].updatedBlockMap[k2] = s2.copy()

In [147]:
def combineParChild(mbm,addr):
    par = mbm[addr].parMap
    chi = mbm[addr].updatedBlockMap
    kill = mbm[addr].parShouldKill
    ckill = mbm[addr].updatedshouldKill
    
    mbm[addr].outMap = op2(par,chi,kill,ckill)
    
    mbm[addr].outEspOff = set()
    mbm[addr].outEbpOff = set()
    
    mbm[addr].outEspOff = updateSet(mbm[addr].espOff,mbm[addr].parEspOff,mbm[addr].parEbpOff,bit_rep = 1)
    mbm[addr].outEbpOff = updateSet(mbm[addr].ebpOff,mbm[addr].parEspOff,mbm[addr].parEbpOff,bit_rep = 1)
    
    if ckill == None:
        mbm[addr].outShouldKill = mbm[addr].parShouldKill
        mbm[addr].outMayKill = mbm[addr].parMayKill 
    else:
        if mbm[addr].parShouldKill != None:
            mbm[addr].outShouldKill = mbm[addr].parShouldKill | ckill
        else :
            mbm[addr].outShouldKill = ckill
         
        if  mbm[addr].parMayKill != None:
            mbm[addr].outMayKill = mbm[addr].parMayKill | ckill
        else:
            mbm[addr].outMayKill = ckill
    

In [148]:
def addFun(mg,mbm,funmap,addr):
    if mbm[addr].funcall:
        fc = traverse_graph(mg,mbm,funmap,mbm[addr].funcall)
        m = op2(mbm[addr].outMap,fc.m,mbm[addr].updatedshouldKill,fc.shouldKill)
        mbm[addr].outMap = m

In [149]:
def process(mg,mbm,funmap,addr):
    updateBlockMap(mbm,addr)
    combineParChild(mbm,addr)
    addFun(mg,mbm,funmap,addr)

In [150]:
def trav(mg,mbm,funmap,addr,vis):
    if addr in vis:
        return 0
    chng = 0
    vis.add(addr)
    
    if mbm[addr].changed:
        chng = 1
        mbm[addr].changed = False
        process(mg,mbm,funmap,addr)
        for naddr in mg.g.adj[addr]:
            relax(addr,naddr,mbm)
    
    for naddr in mg.g.adj[addr]:
        if trav(mg,mbm,funmap,naddr,vis):
            chng = 1
            
    return chng
        

In [151]:
def compute(mg,mbm,addr,f,vis):
    if addr in vis:
        return
    vis.add(addr)
    
    flag = 0
    for naddr in mg.g.adj[addr]:
        flag = 1
        compute(mg,mbm,naddr,f,vis)
    
    if flag == 0:
        (m,x) = op1(mbm[addr].outMap,f.m)
        f.m = m
        if f.shouldKill == None:
            f.shouldKill =  mbm[addr].outShouldKill
        elif mbm[addr].outShouldKill != None:
            f.shouldKill = f.shouldKill & mbm[addr].outShouldKill

In [152]:
def traverse_graph(mg,mbm,funmap,funaddr):
    if funaddr in funmap:
        return funmap[funaddr]
    
    chng = 1
    addr = funaddr
    mbm[funaddr].changed = True
    mbm[funaddr].parEspOff = {('stack',claripy.BVV(0x0,64))}
    mbm[funaddr].parEbpOff = {('base',claripy.BVV(0x0,64))}
    while chng == 1:
        chng = trav(mg,mbm,funmap,funaddr,set())
    
    f = funDet()
    
    compute(mg,mbm,funaddr,f,set())
    funmap[funaddr] = f
    
    return f
    

In [153]:
def filter_glb(m):
    m1 = {}
    for k in m:
        if k[0] == 'glb':
            s = m[k]
            s1 = set()
            for k1 in s:
                if k1[0]=='glb':
                    s1.add(k1)
            m1[k] = s1
    return m1

In [174]:
p = angr.Project("../test/test1/test1.exe",auto_load_libs=False)
fun_addr = p.loader.main_object.get_symbol('main').rebased_addr
cfg = p.analyses.CFGEmulated(starts=[fun_addr],initial_state = p.factory.blank_state())
plot_cfg(cfg, "../test/test1/test1vex", vexinst=True, remove_imports=True, remove_path_terminator=True)
plot_cfg(cfg, "../test/test1/test1asm", asminst=True, remove_imports=True, remove_path_terminator=True)

WARNING | 2022-05-23 20:44:51,285 | cle.loader | The main binary is a position-independent executable. It is being loaded with a base address of 0x400000.


In [175]:
ncfg = newCfg(cfg)
m = find_block_state(cfg)
cm = find_compress_block_state(m)
mbm = generate_myBlock(cm)
funmap = {}
funaddr = p.loader.main_object.get_symbol('fun').rebased_addr
f = traverse_graph(ncfg,mbm,funmap,funaddr)

In [176]:
filter_glb(f.m)

{('glb', 4210720): {('glb', 4210704)},
 ('glb', 4210712): {('glb', 4210704)},
 ('glb', 4210724): {('glb', 4210704)},
 ('glb', 4210716): {('glb', 4210724)}}

In [179]:
p = angr.Project("../test/test2/test2.exe",auto_load_libs=False)
fun_addr = p.loader.main_object.get_symbol('main').rebased_addr
cfg = p.analyses.CFGEmulated(starts=[fun_addr],initial_state = p.factory.blank_state())
plot_cfg(cfg, "../test/test2/test2vex", vexinst=True, remove_imports=True, remove_path_terminator=True)
plot_cfg(cfg, "../test/test2/test2asm", asminst=True, remove_imports=True, remove_path_terminator=True)

WARNING | 2022-05-23 20:46:59,576 | cle.loader | The main binary is a position-independent executable. It is being loaded with a base address of 0x400000.


In [180]:
ncfg = newCfg(cfg)
m = find_block_state(cfg)
cm = find_compress_block_state(m)
mbm = generate_myBlock(cm)
funmap = {}
funaddr = p.loader.main_object.get_symbol('fun').rebased_addr
f = traverse_graph(ncfg,mbm,funmap,funaddr)

In [181]:
filter_glb(f.m)

{('glb', 4210712): {('glb', 4210704), ('glb', 4210716)},
 ('glb', 4210724): {('glb', 4210704), ('glb', 4210716), ('glb', 4210720)},
 ('glb', 4210720): {('glb', 4210704), ('glb', 4210720)}}

In [184]:
p = angr.Project("../test/test3/test3.exe",auto_load_libs=False)
fun_addr = p.loader.main_object.get_symbol('main').rebased_addr
cfg = p.analyses.CFGEmulated(starts=[fun_addr],initial_state = p.factory.blank_state())
plot_cfg(cfg, "../test/test3/test3vex", vexinst=True, remove_imports=True, remove_path_terminator=True)
plot_cfg(cfg, "../test/test3/test3asm", asminst=True, remove_imports=True, remove_path_terminator=True)

WARNING | 2022-05-23 20:57:49,191 | cle.loader | The main binary is a position-independent executable. It is being loaded with a base address of 0x400000.


In [185]:
ncfg = newCfg(cfg)
m = find_block_state(cfg)
cm = find_compress_block_state(m)
mbm = generate_myBlock(cm)
funmap = {}
funaddr = p.loader.main_object.get_symbol('fun').rebased_addr
f = traverse_graph(ncfg,mbm,funmap,funaddr)

In [186]:
filter_glb(f.m)

{('glb', 4210720): {('glb', 4210704), ('glb', 4210728)},
 ('glb', 4210640): set(),
 ('glb', 4210736): {('glb', 4210704)}}

In [187]:
p = angr.Project("../test/test4/test4.exe",auto_load_libs=False)
fun_addr = p.loader.main_object.get_symbol('main').rebased_addr
cfg = p.analyses.CFGEmulated(starts=[fun_addr],initial_state = p.factory.blank_state())
plot_cfg(cfg, "../test/test4/test4vex", vexinst=True, remove_imports=True, remove_path_terminator=True)
plot_cfg(cfg, "../test/test4/test4asm", asminst=True, remove_imports=True, remove_path_terminator=True)

WARNING | 2022-05-23 20:57:54,224 | cle.loader | The main binary is a position-independent executable. It is being loaded with a base address of 0x400000.


In [188]:
ncfg = newCfg(cfg)
m = find_block_state(cfg)
cm = find_compress_block_state(m)
mbm = generate_myBlock(cm)
funmap = {}
funaddr = p.loader.main_object.get_symbol('fun').rebased_addr
f = traverse_graph(ncfg,mbm,funmap,funaddr)

In [189]:
filter_glb(f.m)

{('glb', 4210640): set(),
 ('glb', 4210736): {('glb', 4210704), ('glb', 4210736)},
 ('glb', 4210744): {('glb', 4210704), ('glb', 4210736), ('glb', 4210744)},
 ('glb', 4210720): {('glb', 4210704), ('glb', 4210736), ('glb', 4210744)}}